In [1]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from func_timeout import func_timeout, FunctionTimedOut
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import urlsplit
from func_timeout import func_timeout, FunctionTimedOut
import pickle
import requests
import pandas_dedupe

In [2]:
import logging

logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [3]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

# Proxy Pool

In [524]:
with open("proxy_new-800.txt", "rb") as fp:   
    proxy_new = pickle.load(fp)
login = 'usr14362297'
password = 'ps143621582973974'
proxy_pool = cycle(proxy_new[0:100])

In [525]:
with open("rev_links.pkl", "rb") as fp:  
    rev_links = pickle.load(fp)

# Metadata

In [4]:
s = cfscrape.create_scraper()
scraper = cfscrape.create_scraper(sess=s)

In [526]:
proxy = next(proxy_pool)
go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
s = cfscrape.create_scraper()
s.proxies = go_proxy
scraper = cfscrape.create_scraper(sess=s)
url = rev_links[600]
print(url)

scraped_html=scraper.get(url).content
html = etree.HTML(scraped_html)

https://www.mezcalreviews.com/mezcal/los-nahuales-special-edition-no-1/


In [ ]:
page_dict = {}
for i in range(1,20):
    key = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/th/text()'.format(i))
    if key and 'rating' in key[0].lower():
        value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td//i//@class'.format(i))
        reviews = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td/text()'.format(i))
        page_dict[key[0]] = []
        page_dict[key[0]].extend(value)
        page_dict[key[0]].extend(reviews)
    
    elif key and 'cost' in key[0].lower():
        value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td/a//text()'.format(i))
        page_dict[key[0]] = value
    elif key:
        value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td//text()'.format(i))
        page_dict[key[0]] = value

In [172]:
#bad_idx = table[table['metadata'].isna()].index.to_list()

In [170]:
for page in bad_idx:
    print('Page: {}'.format(page+1))
    
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    url = rev_links[page]
    
    try:
        def get_page_meta():
            scraped_html=scraper.get(url).content
            html = etree.HTML(scraped_html)
            page_dict = {}
            for i in range(1,20):
                key = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/th/text()'.format(i))
                if key and 'rating' in key[0].lower():
                    value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td//i//@class'.format(i))
                    reviews = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td/text()'.format(i))
                    page_dict[key[0]] = []
                    page_dict[key[0]].extend(value)
                    page_dict[key[0]].extend(reviews)

                elif key and 'cost' in key[0].lower():
                    value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td/a//text()'.format(i))
                    page_dict[key[0]] = value
                elif key:
                    value = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/div[2]/table/tbody/tr[{}]/td//text()'.format(i))
                    page_dict[key[0]] = value

            img_link = html.xpath('/html/body/section[4]/div/div/div[2]/div[1]//img//@srcset')[-1].split(', ')[-1].split(' ')[0]
            page_dict['img_link'] = img_link
            page_dict['site_link'] = url

            table['metadata'].loc[page] = [page_dict]
            print ('Scraped')
        
        try:
            func_timeout(60, get_page_meta)
            
        except FunctionTimedOut:
            print('Timeout')
            continue
    
    except:
        print('Smth went wrong')

Page: 224
Scraped


In [258]:
df = pd.DataFrame([])
log_dict = {}

In [260]:
for i in range(table.shape[0]):
    #print(i)
    
    if type(table['metadata'][i][0]['Rating:'])!=float and 'no reviews' not in table['metadata'][i][0]['Rating:'][0].lower():
        rating = ', '.join(table['metadata'][i][0]['Rating:'])
        one_star = re.findall(r'fa fa-star(?!-)', rating)
        half_star = re.findall(r'fa fa-star-o', rating)
        rate = 1*len(one_star)+0.5*len(half_star)
        reviews = re.findall(r'(?<=\().+rev.+(?=\))', rating)
        table['metadata'][i][0]['Rating:'] = rate

        if reviews:
            table['metadata'][i][0]['Reviews'] = reviews[0]
    
    if 'Category:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Category:']) == list:
            table['metadata'][i][0]['Category:'] = ', '.join(table['metadata'][i][0]['Category:'])
            
    if 'Raicillero:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Raicillero:']) == list:
            table['metadata'][i][0]['Raicillero:'] = ', '.join(table['metadata'][i][0]['Raicillero:'])
            
    if 'Sotolero:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Sotolero:']) == list:
            table['metadata'][i][0]['Sotolero:'] = ', '.join(table['metadata'][i][0]['Sotolero:'])
            
    if 'State:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['State:']) == list:
            table['metadata'][i][0]['State:'] = ', '.join(table['metadata'][i][0]['State:']) 
    
    if 'Town:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Town:']) == list:
            table['metadata'][i][0]['Town:'] = ', '.join(table['metadata'][i][0]['Town:'])
  
    if 'Style:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Style:']) == list:
            table['metadata'][i][0]['Style:'] = ', '.join(table['metadata'][i][0]['Style:'])

    
    if 'NOM:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['NOM:']) == list:
            table['metadata'][i][0]['NOM:'] = ', '.join(table['metadata'][i][0]['NOM:'])
   
    if 'Distillation:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Distillation:']) == list:
            table['metadata'][i][0]['Distillation:'] = ', '.join(table['metadata'][i][0]['Distillation:'])
    
    if 'Agave:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Agave:']) == list:
            table['metadata'][i][0]['Agave:'] = ', '.join(table['metadata'][i][0]['Agave:'])
  
    if 'Grind:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Grind:']) == list:
            table['metadata'][i][0]['Grind:'] = ', '.join(table['metadata'][i][0]['Grind:'])
  
    if 'Mezcalero:' in table['metadata'][i][0].keys():
        if type( table['metadata'][i][0]['Mezcalero:']) == list:
            table['metadata'][i][0]['Mezcalero:'] = ', '.join(table['metadata'][i][0]['Mezcalero:'])
    
    if 'Cost:' in table['metadata'][i][0].keys():
        cost = table['metadata'][i][0]['Cost:']
        cost = len(re.findall(r'\$', str(cost)))
        table['metadata'][i][0]['Cost:'] = cost
        
    try:
        row = pd.DataFrame(table['metadata'][i][0], index=[i])
        df = pd.concat([df, row], axis=0, ignore_index=False)
        #print('GOOD')
        #log_dict[i]='GOOD'
    except:
        log_dict[i]='WRONG'
        #print('WRONG')
        continue

In [476]:
for i in idx : #range(df.shape[0])
    print('Page: {}'.format(i+1))
    
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    url = df['site_link'][i]
    
    try:
        def get_page_meta():
            scraped_html=scraper.get(url).content
            html = etree.HTML(scraped_html)
            bot_name = html.xpath('/html/body/section[4]/div/div/div[1]/div[1]/h1/text()')
            if bot_name:
                df['Bottle Name'].loc[i] = bot_name[0]
                print (bot_name[0])

        try:
            func_timeout(60, get_page_meta)

        except FunctionTimedOut:
            print('Timeout')
            continue
    
    except:
           print('Smth went wrong')

Page: 7
Don Amado Tobala & Bicuishe
Page: 34
Xicaru Pechuga Mole
Page: 47
Cenizo Colonial Hacienda Dolores
Page: 53
Rezpiral Tequilana
Page: 61
Vago Bien Picado
Page: 107
Real Minero Barril, Madrecuishe
Page: 164
Cuentacuentos Seemanniana
Page: 207
Riazuleño Contemporaneo
Page: 364
Santo de Piedra Mezcal
Page: 629
Bozal Cuixe
Page: 634
Siete Misterios Doba-Yej
Page: 653
Viejo Indecente Espadin
Page: 761
Mezcalero No. 13
Page: 764
Nuestra Soledad La Compania Ejutla


In [482]:
cols = []
for i in df.columns:
    i = re.sub(':','',i)
    i = re.sub('img_link','Image Link',i)
    i = re.sub('site_link','Review link',i)
    cols.append(i)

In [484]:
df.columns = cols

In [486]:
with open("rev_table_processed.pkl", "wb") as fp:  
    pickle.dump(df, fp)

# Retailers

In [456]:
old_tequila = pd.DataFrame([], index = range(1,633))

In [457]:
old_tequila['bot_link'] = np.nan
old_tequila['bot_name'] = np.nan
old_tequila['bot_brand'] = np.nan
old_tequila['bot_price'] = np.nan
old_tequila['bot_rating'] = np.nan
old_tequila['bot_num_reviews'] = np.nan
old_tequila['bot_review'] = np.nan
old_tequila['bot_description'] = np.nan

In [458]:
for page in range(1,8):
    print('Page: {}'.format(page))
    
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    url = 'https://www.oldtowntequila.com/mezcal/?sort=newest&limit=100&mode=4&page={}'.format(page)
    
   
    scraped_html=scraper.get(url).content
    html = etree.HTML(scraped_html)
    
    for i in range(1,101):
        index = (page-1)*100+i
        link = html.xpath('//*[@id="product-listing-container"]/form/ul/li[{}]/article/figure/a/@href'.format(i))
        if link:
            old_tequila['bot_link'].loc[index] = link[0]
        
        bottle_name = html.xpath('//*[@id="product-listing-container"]/form/ul/li[{}]/article/div[1]/h4/a/text()'.format(i))
        if bottle_name:
            old_tequila['bot_name'].loc[index] = bottle_name[0]
        
        brand = html.xpath('//*[@id="product-listing-container"]/form/ul/li[{}]/article/div[1]/p/text()'.format(i))
        if brand:
            old_tequila['bot_brand'].loc[index] = brand[0]
        
        price = html.xpath('//*[@id="product-listing-container"]/form/ul/li[{}]/article/div[1]/div[1]/div[2]/span[3]/text()'.format(i))
        if price:
            old_tequila['bot_price'].loc[index] = price[0]


Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7


### reviews

In [6]:
old_tequila['Image_OT'] = np.nan

In [7]:
for n in range(1, old_tequila.shape[0]+1):
    if n%50==0:
        print('Page: {}'.format(n))
    
    sleep(randint(3,7))
    url = old_tequila['bot_link'][n]
    
    try:
        scraped_html=scraper.get(url).content
        html = etree.HTML(scraped_html)

        rating = html.xpath('//*[@class="productView-rating"]//*[starts-with(@class,"icon icon--rating")]//@class')
        old_tequila['bot_rating'].loc[n] = rating
        
        description = html.xpath('//*[starts-with(@class, "productView-description-tabContent")]//text()')
        old_tequila['bot_description'].loc[n] = description

        num_reviews = len(html.xpath('//*[@class="productReviews-list"]//li'))
        old_tequila['bot_num_reviews'].loc[n] = num_reviews
        
        img = html.xpath('//*[@class="productView-detailsWrapper"]//img//@srcset')[-1].split(', ')[-1].split(' ')[0]
        old_tequila['Image_OT'].loc[n] = img
        if n%30==0:
            print(img)
        
        if num_reviews>0:
            bot_reviews = {}
            for i in range(1, num_reviews+1):
                rev_name = 'Review_' + str(i)
                bot_reviews[rev_name] = {}
                rev_title = html.xpath('//*[@class="productReviews-list"]/li[{}]//*[@class="productReview-title"]/text()'.format(i))
                rev_body = html.xpath('//*[@class="productReviews-list"]/li[{}]//*[@class="productReview-body"]/text()'.format(i))
                rev_author = html.xpath('//*[@class="productReviews-list"]/li[{}]//*[@class="productReview-author"]/text()'.format(i))
                rev_rating = html.xpath('//*[@class="productReviews-list"]/li[{}]//*[starts-with(@class,"icon icon--rating")]//@class'.format(i))

                bot_reviews[rev_name]['rev_title'] = rev_title
                bot_reviews[rev_name]['rev_body'] = rev_body
                bot_reviews[rev_name]['rev_author'] = rev_author
                bot_reviews[rev_name]['rev_rating'] = rev_rating

            old_tequila['bot_review'].loc[n] = [bot_reviews]
    except:
        continue

https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8979/13006/MestizaNegraBottle__36349.1576018465.jpg?c=2
Page: 50
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8858/12797/agave_macho_espadin_madrecuishe_joven_mezcal_750ml__31513.1570224997.png?c=2
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8673/12520/mezcales_de_leyenda_mini_set__91235.1562974720.png?c=2
Page: 100
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8496/12277/el_rey_zapoteco_joven_tobala_mezcal__86731.1554747479.png?c=2
Page: 150
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8344/12055/mezcalosfera_espadin_jabali__82624.1548456344.png?c=2
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/8080/11551/se_busca_product-joven__44626.1538179948.png?c=2
Page: 200
https://cdn11.bigcommerce.com/s-u9ww3di/images/stencil/2560w/products/7910/11271/lalocura_tobishe_mezcal__27851.1531119980.png?c=2
Pa

## Cleaning

In [15]:
#with open("old_tequila_raw.pkl", "wb") as fp:  
    #pickle.dump(old_tequila, fp)

In [128]:
with open("old_tequila_raw.pkl", "rb") as fp:
    old_tequila = pickle.load(fp)

In [129]:
rev_table=pd.DataFrame([])
for i in idx:
    count = 0
    dict_page = {}
    
    for key, main_value in old_tequila['bot_review'][i][0].items():
        count+=1
        title = main_value['rev_title']
        body = main_value['rev_body']
        review_text = '. '.join(title + body)
        review_text = re.sub(r'\n|\r','', review_text)
        main_value['rev_body'] = review_text
        main_value['rev_title'] = main_value['rev_title'][0]

        rating = main_value['rev_rating']
        if type(rating) == list:
            rating = ', '.join(rating)
            one_star = re.findall(r'ratingFull', rating)
            main_value['rev_rating'] = len(one_star)
        elif type(rating) == int or type(rating) == float:
            main_value['rev_rating'] = rating
        
        main_value['rev_author'] = re.sub(r'\n|\r|  +','', main_value['rev_author'][0])

        dict_page['rev_body_{}'.format(count)] = main_value['rev_body']
        dict_page['rev_rating_{}'.format(count)] = main_value['rev_rating']
        dict_page['rev_author_date_{}'.format(count)] = main_value['rev_author']
    
    row = pd.DataFrame(dict_page, index=[i])
    rev_table = pd.concat([rev_table, row], axis=0)

In [130]:
for i in range(old_tequila.shape[0]):
    try:

        rating = ''.join(old_tequila['bot_rating'][i])
        one_star = re.findall(r'ratingFull', rating)
        rating = len(one_star)
        old_tequila['bot_rating'][i] = rating
    except:
        continue

In [131]:
cols = ['bot_link','bot_name','bot_brand','bot_price','bot_rating','bot_description','bot_num_reviews','Image_OT']
old_tequila = pd.merge(old_tequila[cols].reset_index(), rev_table.reset_index(), on='index',how='left').drop(columns=['index'])

In [86]:
old_tequila['bot_description'] = old_tequila['bot_description'].fillna('')
for i in range(old_tequila.shape[0]):
    descr = ', '.join(old_tequila['bot_description'][i])
    descr = re.sub('\n  +, |, \n  +','',descr)
    descr = re.sub(', \r\n,',';', descr)
    descr = re.sub(': , ',': ', descr)
    descr = re.sub('\xa0','', descr)
    descr = re.sub('\r\n;|\r\n,','', descr)
      
    old_tequila['bot_description'][i] = descr

In [87]:
old_tequila.head(1)

,bot_link,bot_name,bot_brand,bot_price,bot_rating,bot_description,bot_num_reviews,Image_OT,rev_body_1,rev_rating_1,rev_author_date_1,rev_body_2,rev_rating_2,rev_author_date_2,rev_body_3,rev_rating_3,rev_author_date_3,rev_body_4,rev_rating_4,rev_author_date_4,rev_body_5,rev_rating_5,rev_author_date_5,rev_body_6,rev_rating_6,rev_author_date_6,rev_body_7,rev_rating_7,rev_author_date_7,rev_body_8,rev_rating_8,rev_author_date_8,rev_body_9,rev_rating_9,rev_author_date_9
0,https://www.oldtowntequila.com/400-conejos-esp...,400 Conejos Espadin Mezcal Artesanal,400 Conejos,$27.99,3,400 Conejos Espadin Mezcal Artesanal Joven 750...,1,https://cdn11.bigcommerce.com/s-u9ww3di/images...,400 Conejos Mezcal. Has a little bite well pri...,3.0,Posted by Thomas on May 2nd 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
with open("old_tequila_processed.pkl", "wb") as fp:  
    pickle.dump(old_tequila, fp)

# drizly.com

In [527]:
proxy = next(proxy_pool)
go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
s = cfscrape.create_scraper()
s.proxies = go_proxy
scraper = cfscrape.create_scraper(sess=s)

In [532]:
scraped_html=scraper.get(url).content
html = etree.HTML(scraped_html)

In [596]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome('/Users/macbook/Downloads/chromedriver')

## links

In [606]:
url = 'https://drizly.com/liquor/mezcal/c86/page2'
browser.get(url)

In [ ]:
links = []
for i in range(1,14):
    url = 'https://drizly.com/liquor/mezcal/c86/page{}'.format(i)
    sleep(randint(5,10))
    print('#{} out of 13'.format(i))
    
    browser.get(url)
    sleep(randint(5,10))
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    count = 0
    for anchor in soup.find_all("a"):
        if "href" in anchor.attrs and '/liquor/mezcal' in anchor.attrs['href'] and 'c86' not in anchor.attrs['href']:
            count+=1
            print('#{}. link:{}'.format(count, anchor.attrs['href']))
            links.append(anchor.attrs['href'])

In [609]:
len(links)

300

In [611]:
links = ['https://drizly.com' + x for x in links]

In [ ]:
'https://drizly.com/liquor/mezcal/del-maguey-mezcal-tobala/p28135'

### In the page

In [751]:
url = links[4]
#url = 'https://drizly.com/liquor/mezcal/casamigos-joven-mezcal/p84690'
browser.get(url)
print(url)

https://drizly.com/liquor/mezcal/banhez-mezcal/p72987


In [752]:
# name
#sleep(3)
path = '/html/body/div[5]/main/section[1]/div[1]/div[2]/h1'
browser.find_element_by_xpath(path).text

'Banhez Mezcal'

In [753]:
# price
path = '/html/body/div[5]/main/section[1]/div[2]/fieldset/div'
browser.find_element_by_xpath(path).text

'750.0ml bottle - from $32.99 \n1.0L bottle - from $37.99 '

In [754]:
# reviews WITH TRY
path = '/html/body/div[5]/main/section[1]/div[1]/div[2]/button/span[1]/div'
browser.find_element_by_xpath(path).text

'5(1 Reviews)'

In [755]:
# ABV, town
path = '/html/body/div[5]/main/section[1]/div[1]/div[2]/p'
browser.find_element_by_xpath(path).text

'Mezcal / Mexico'

In [756]:
# description
path = '/html/body/div[5]/main/section[3]/p'
browser.find_element_by_xpath(path).text

'The smoke is there at first but ultimately it plays a background role to the gorgeous fruit driven aromas of lime rind, banana, to the more tropical presence of star fruit and papaya. Very terroir driven, as all good mezcal should be, the beautiful integration of elements on the palate are accented by peppery flavors that waken the senses.'

In [757]:
# image
path = '/html/body/div[5]/main/section[1]/div[1]/div[1]/img'
browser.find_element_by_xpath(path).get_attribute('src')

'https://products1.imgix.drizly.com/ci-banhez-mezcal-f95d2e1daa80d5c4.png?auto=format%2Ccompress&fm=jpg&q=20'

In [758]:
# reviews
# click reviews
path = '//*[@id="DetailsAndReviews"]/div[1]/div/div/div/button[2]/span[1]/span/span/p'
browser.find_element_by_xpath(path).click()

+ check if li[1] works for one review

In [763]:
# number of reviews
#path = '//*[@id="DetailsAndReviews"]/section/ul'
rev_num = browser.find_element_by_id("DetailsAndReviews").find_elements_by_tag_name("li")
rev_num = len(rev_num)
rev_num

1

In [760]:
# stars for review
path = '//*[@id="DetailsAndReviews"]/section/ul/li[1]/article/span/span[2]'
rev_stars = browser.find_element_by_xpath(path).get_attribute('innerHTML')
stars = len(re.findall('M12 17.27L18.18', rev_stars))
stars

5

In [761]:
# review text
path = '//*[@id="DetailsAndReviews"]/section/ul/li[1]/article/span'
browser.find_element_by_xpath(path).text#.split('Verified Buyer')

'Octave\nMay 1, 2018\nPerfect for sipping or in cocktails, with a delicate vegetal profile. Unbeatable at that price. I love it and you will too!'

In [762]:
# review pages
path = '//*[@id="DetailsAndReviews"]/section/span[2]/button/span[1]'
browser.find_element_by_xpath(path).text

'1'

## loop

In [765]:
drizly_data = pd.DataFrame({"links":links,
                           'metadata':[np.nan]*len(links)})
drizly_data.shape 

(300, 2)

In [787]:
for i in range(100,301): #
    
    url = drizly_data["links"][i]
    print('#{} out of 300'.format(i))
    print(url)
    

    page_data = {}
    name = np.nan
    price = np.nan
    rate_review = np.nan
    abv_town = np.nan
    description = np.nan
    image = np.nan
    rev_num = np.nan
    rev_page_count = np.nan
    page_rev_data = np.nan
    
    
    try:
        browser.get(url)
        sleep(randint(10,20))
        name = browser.find_element_by_xpath('/html/body/div[5]/main/section[1]/div[1]/div[2]/h1').text
        print('Got name')
    except:
        print('NO name')
        pass
        
    try:
        price = browser.find_element_by_xpath('/html/body/div[5]/main/section[1]/div[2]/fieldset/div').text
        print('Got price')
    except:
        print('NO price')
        pass
        
    try:
        rate_review = browser.find_element_by_xpath('/html/body/div[5]/main/section[1]/div[1]/div[2]/button/span[1]/div').text
        print('Got rate_review')
    except:
        print('NO rate_review')
        pass
        
    try:
        abv_town = browser.find_element_by_xpath('/html/body/div[5]/main/section[1]/div[1]/div[2]/p').text
        print('Got abv_town')
    except:
        print('NO abv_town')
        pass
        
    try:
        description = browser.find_element_by_xpath('/html/body/div[5]/main/section[3]/p').text
        print('Got description')
    except:
        print('NO description')
        pass
   
    try:
        image = browser.find_element_by_xpath('/html/body/div[5]/main/section[1]/div[1]/div[1]/img').get_attribute('src')
        print('Got image')
    except:
        print('NO image')
        pass
    
    try:
        page_rev_data = {}
        count=0
        # click review
        browser.find_element_by_xpath('//*[@id="DetailsAndReviews"]/div[1]/div/div/div/button[2]/span[1]/span/span/p').click()
        print('Clicked on reviews')
        
        # number of reviews
        rev_num = browser.find_element_by_id("DetailsAndReviews").find_elements_by_tag_name("li")
        rev_num = len(rev_num)
        
        for n in range(1, rev_num+1):
            count+=1
    
            # stars for review
            path = '//*[@id="DetailsAndReviews"]/section/ul/li[{}]/article/span/span[2]'.format(n)
            rev_stars = browser.find_element_by_xpath(path).get_attribute('innerHTML')
            stars = len(re.findall('M12 17.27L18.18', rev_stars))
            stars_key = '{}_rev_stars'.format(count)
            page_rev_data[stars_key] = stars
            
            # review text
            path = '//*[@id="DetailsAndReviews"]/section/ul/li[{}]/article/span'.format(n)
            text = browser.find_element_by_xpath(path).text
            text_key = '{}_rev_text'.format(count)
            page_rev_data[text_key] = text
        
        # review pages
        rev_page_count = browser.find_element_by_xpath('//*[@id="DetailsAndReviews"]/section/span[2]/button/span[1]').text
        print('Got reviews data')
    
    except:
        print('NO reviews data')
        pass
    
    page_data['name'] = name
    page_data['price'] = price
    page_data['rate_review'] = rate_review
    page_data['abv_town'] = abv_town
    page_data['description'] = description
    page_data['image'] = image
    page_data['rev_num'] = rev_num
    page_data['rev_page_count'] = rev_page_count
    page_data['page_rev_data'] = page_rev_data
    
    drizly_data['metadata'].loc[i] = [page_data]

#100 out of 300
https://drizly.com/liquor/mezcal/paquera-mezcal-espadin/p88004
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#101 out of 300
https://drizly.com/liquor/mezcal/el-mero-mero-espadin-mezcal/p91893
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#102 out of 300
https://drizly.com/liquor/mezcal/los-javis-espadin/p63046
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#103 out of 300
https://drizly.com/liquor/mezcal/bozal-sacatoro-mezcal/p98062
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#104 out of 300
https://drizly.com/liquor/mezcal/el-jolgorio-espadin/p65734
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#105 out of 300
https://drizly.com/liquor/mezcal/nuestra-soledad-la-compania-ejutla/p64446
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO 

Got name
Got price
Got rate_review
Got abv_town
Got description
Got image
Clicked on reviews
Got reviews data
#149 out of 300
https://drizly.com/liquor/mezcal/fidencio-pechuga-mezcal/p11056
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#150 out of 300
https://drizly.com/liquor/mezcal/mestizo-mezcal-reposado/p16451
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#151 out of 300
https://drizly.com/liquor/mezcal/the-lost-explorer-mezcal/p65704
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#152 out of 300
https://drizly.com/liquor/mezcal/el-jolgorio-cuixe/p65743
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#153 out of 300
https://drizly.com/liquor/mezcal/don-amado-mezcal-arroqueno/p68573
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#154 out of 300
https://drizly.com/liquor/mezcal

Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#197 out of 300
https://drizly.com/liquor/mezcal/bozal-ancestral-mezcal/p68566
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#198 out of 300
https://drizly.com/liquor/mezcal/mezcal-vago-tepeztate/p70775
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#199 out of 300
https://drizly.com/liquor/mezcal/la-nina-del-mezcal-primario-mezcal-joven/p81247
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#200 out of 300
https://drizly.com/liquor/mezcal/ray-campero-mezcal-joven/p82614
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#201 out of 300
https://drizly.com/liquor/mezcal/maximo-joven-mezcal/p89221
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#202 out of 300
https://drizly.com/liquor/mezcal/erstwhil

Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#245 out of 300
https://drizly.com/liquor/mezcal/wahaka-guajolote-con-fruta/p65560
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#246 out of 300
https://drizly.com/liquor/mezcal/escorpion-silver-barril/p65782
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#247 out of 300
https://drizly.com/liquor/mezcal/el-rey-zapoteco-mezcal-anejo/p68539
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#248 out of 300
https://drizly.com/liquor/mezcal/convite-coyote-mezcal/p86471
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#249 out of 300
https://drizly.com/liquor/mezcal/jaral-de-berrio-mezcal/p6056
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#250 out of 300
https://drizly.com/liquor/mezcal/del-maguey-wild-p

Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#294 out of 300
https://drizly.com/liquor/mezcal/bozal-pechuga-mezcal/p68563
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#295 out of 300
https://drizly.com/liquor/mezcal/mestizo-mezcal-joven/p68567
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#296 out of 300
https://drizly.com/liquor/mezcal/don-amado-mezcal-largo/p68574
Got name
Got price
NO rate_review
Got abv_town
Got description
Got image
NO reviews data
#297 out of 300
https://drizly.com/liquor/mezcal/don-amando-mezcal-anejo/p85256
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#298 out of 300
https://drizly.com/liquor/mezcal/clase-azul-mezcal-plata/p86167
Got name
Got price
NO rate_review
Got abv_town
NO description
Got image
NO reviews data
#299 out of 300
https://drizly.com/liquor/mezcal/don-mateo-mezcal-alto/p93221

KeyError: 300

In [818]:
reviews_data = pd.DataFrame([])
for i in range(drizly_data.shape[0]):
    row = pd.DataFrame(drizly_data['metadata'].loc[i][0]['page_rev_data'], index=[i])
    reviews_data = pd.concat([reviews_data, row], axis=0, ignore_index=True)

In [827]:
name = drizly_data['metadata'].loc[111][0]['name']
price = drizly_data['metadata'].loc[111][0]['price']
rate_review = drizly_data['metadata'].loc[111][0]['rate_review']
abv_town = drizly_data['metadata'].loc[111][0]['abv_town']
description = drizly_data['metadata'].loc[111][0]['description']
image = drizly_data['metadata'].loc[111][0]['image']
rev_num = drizly_data['metadata'].loc[111][0]['rev_num']
rev_page_count = drizly_data['metadata'].loc[111][0]['rev_page_count']

In [830]:
drizly_data['name'] = np.nan
drizly_data['price'] = np.nan
drizly_data['rate_review'] = np.nan
drizly_data['abv_town'] = np.nan
drizly_data['description'] = np.nan
drizly_data['image'] = np.nan
drizly_data['rev_num'] = np.nan
drizly_data['rev_page_count'] = np.nan

In [831]:
for i in range(drizly_data.shape[0]):
    drizly_data['name'].loc[i] = drizly_data['metadata'].loc[i][0]['name']
    drizly_data['price'].loc[i] = drizly_data['metadata'].loc[i][0]['price']
    drizly_data['rate_review'].loc[i] = drizly_data['metadata'].loc[i][0]['rate_review']
    drizly_data['abv_town'].loc[i] = drizly_data['metadata'].loc[i][0]['abv_town']
    drizly_data['description'].loc[i] = drizly_data['metadata'].loc[i][0]['description']
    drizly_data['image'].loc[i] = drizly_data['metadata'].loc[i][0]['image']
    drizly_data['rev_num'].loc[i] = drizly_data['metadata'].loc[i][0]['rev_num']
    drizly_data['rev_page_count'].loc[i] = drizly_data['metadata'].loc[i][0]['rev_page_count']

In [834]:
drizly_data = pd.concat([drizly_data, reviews_data], axis=1)

### price

In [852]:
drizly_data['price-one'] = drizly_data['price'].str.split(' \n', expand=True)[0].str.split(' bottle - from ', expand=True)[1]
drizly_data['price-one-bottle'] = drizly_data['price'].str.split(' \n', expand=True)[0].str.split(' bottle - from ', expand=True)[0]

In [853]:
drizly_data['price-two'] = drizly_data['price'].str.split(' \n', expand=True)[1].str.split(' bottle - from ', expand=True)[1]
drizly_data['price-two-bottle'] = drizly_data['price'].str.split(' \n', expand=True)[1].str.split(' bottle - from ', expand=True)[0]

### rating

In [961]:
drizly_data['rating'] = drizly_data['rate_review'].str.extract(r'(.+(?=\(.+ Revie\w+\)))')

In [872]:
drizly_data['given_rev_num'] = drizly_data['rate_review'].str.extract(r'(\d+(?= Revie))').astype(float)
# check num of reviews
(drizly_data['given_rev_num'] - drizly_data['rev_num']).sum()

### ABV / Town

In [883]:
drizly_data['ABV'] = drizly_data['abv_town'].str.extract(r'(\d+%(?= ABV))')

In [886]:
drizly_data['Town'] = drizly_data['abv_town'].str.extract(r'((?<=\/ )\w.+$)')

### Reviews Date, Author, Buyer

In [949]:
for col in drizly_data.columns:
    if 'rev_text' in col:
        
        date = drizly_data[col].str.extract(r'((?<=\n).+(?=\n))')
        col_name = col[:-4]+'Date'
        drizly_data[col_name] = date
        
        name = drizly_data[col].str.extract(r'(^.+[^Verified Buyer](?=\n.+\d)|.+(?=Verified Buyer))')
        col_name = col[:-4]+'Reviewer Name'
        drizly_data[col_name] = name
        
        text = drizly_data[col].str.extract(r'((?<=\d\n).+$)')
        col_name = col[:-4]+'Text'
        drizly_data[col_name] = text
        
        ver_buyer = drizly_data[col].str.lower().str.extract(r'(verified buyer)')
        col_name = col[:-4]+'Verified Buyer'
        drizly_data[col_name] = ver_buyer

### Prearing for GS

In [964]:
sorted_cols = []
for i in range(1,11):
    for col in drizly_data.columns:
        if str(i)+'_rev' in col and 'rev_text' not in col:
            sorted_cols.append(col)

In [965]:
first_cols = ['name', 'price-one','price-one-bottle', 'price-two', 'price-two-bottle', 
              'description', 'rating', 'rev_num',  'ABV', 'Town','links', 'image']
first_cols.extend(sorted_cols)       

In [970]:
to_sheets = drizly_data[first_cols].fillna('')
wks_name = 'Drizly'
cell_of_start_df = 'A1'
d2g.upload(to_sheets,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=False)

<Worksheet 'Drizly' id:1625297467>

In [971]:
with open("drizly_data_raw.pkl", "wb") as fp:  
    pickle.dump(drizly_data, fp)

### CHECK
+ 6 pack bottles
+ price-bottle
+ calculate rating

# mezcalia

In [1298]:
#with open('mezcalia_meta.pkl', 'wb') as f:
   # pickle.dump(mezcalia_meta, f)

In [1299]:
with open("mezcalia_meta.pkl", "rb") as f:  
    mezcalia_meta = pickle.load(f)

In [995]:
links = []
for i in range(1,18):
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    
    url='https://mezcalia.com/tienda-mezcal/page/{}'.format(i)
    scraped_html=scraper.get(url).content
    html = etree.HTML(scraped_html)
    page_links = html.xpath('//*[@class="ast-loop-product__link"]//@href')
    links.extend(page_links)

In [996]:
len(links)

321

### scraping pages

In [1073]:
mezcalia_df = pd.DataFrame(links, columns=['website_link'])
mezcalia_df.shape

(321, 1)

In [1074]:
mezcalia_df['bottle_name'] = np.nan
mezcalia_df['bottle_price'] = np.nan
mezcalia_df['etiquetas'] = np.nan
mezcalia_df['short_description'] = np.nan
mezcalia_df['description'] = np.nan

In [1109]:
for i in range(36, mezcalia_df.shape[0]):
    print('#{} out of 320'.format(i))
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    
    url = mezcalia_df['website_link'].loc[i]
    print(url)
    
    def get_page_meta():
        scraped_html=scraper.get(url).content
        html = etree.HTML(scraped_html)

        name = html.xpath('//*[@class="product_title entry-title"]//text()')
        mezcalia_df['bottle_name'].loc[i] = name[0]

        price = html.xpath('//*[starts-with(@id,"product")]/div[2]/p//text()') 
        price = re.sub('\xa0', ' ', ''.join(price))
        mezcalia_df['bottle_price'].loc[i] = price

        etiquetas = html.xpath('//*[@class="tagged_as"]//text()') 
        etiquetas = ''.join(etiquetas)
        mezcalia_df['etiquetas'].loc[i] = etiquetas

        short_description = html.xpath('//*[@class="woocommerce-product-details__short-description"]//text()') 
        if short_description:
            short_description = re.sub('\xa0', '', ' '.join(short_description))
            mezcalia_df['short_description'].loc[i] = short_description

        description = html.xpath('//*[@class="woocommerce-tabs wc-tabs-wrapper"]//text()') 
        if description:
            description = re.sub('\xa0', '', ' '.join(description))
            mezcalia_df['description'].loc[i] = description
    
    try:
        func_timeout(60, get_page_meta)
            
    except FunctionTimedOut:
        print('Timeout')
        continue

#36 out of 320
https://mezcalia.com/producto/mezcal-titlan-jabali/
#37 out of 320
https://mezcalia.com/producto/mezcal-convite-coyote/
#38 out of 320
https://mezcalia.com/producto/mezcal-convite-espadin-madrecuixe/
Timeout
#39 out of 320
https://mezcalia.com/producto/mezcal-rio-revuelto-espadin/
#40 out of 320
https://mezcalia.com/producto/augurio-mezcal-espadin/
#41 out of 320
https://mezcalia.com/producto/mezcal-lalocura-tobala/
#42 out of 320
https://mezcalia.com/producto/kit-memorable-arroqueno/
#43 out of 320
https://mezcalia.com/producto/crema-de-mezcal-madrona-coco/
#44 out of 320
https://mezcalia.com/producto/crema-de-mezcal-madrona-beso-oaxaqueno/
#45 out of 320
https://mezcalia.com/producto/mezcal-yuliia-maracuya/
#46 out of 320
https://mezcalia.com/producto/mezcal-yuliia-jamaica/
#47 out of 320
https://mezcalia.com/producto/mezcal-local-pechuga-de-agave/
#48 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-mezcal-convite-jabali/
#49 out of 320
https://mezcalia.com/

#141 out of 320
https://mezcalia.com/producto/cata-de-mezcal-cata-esencial/
#142 out of 320
https://mezcalia.com/producto/mezcal-coyota-pedrosana/
#143 out of 320
https://mezcalia.com/producto/mezcal-quitacalzon-ensamble-espadin-y-tobala-copy-copy/
#144 out of 320
https://mezcalia.com/producto/mezcal-quitacalzon-ensamble-espadin-y-tobala-copy/
#145 out of 320
https://mezcalia.com/producto/mezcal-quitacalzon-ensamble-espadin-y-tobala/
#146 out of 320
https://mezcalia.com/producto/mezcal-lalocura-espadin-copy/
#147 out of 320
https://mezcalia.com/producto/mezcal-cuero-viejo-joven-cenizo/
#148 out of 320
https://mezcalia.com/producto/mezcal-bosscal-joven-cenizo-copy/
#149 out of 320
https://mezcalia.com/producto/mezcal-sertres-espadin-copy-copy/
#150 out of 320
https://mezcalia.com/producto/mezcal-cuish-tepextate/
#151 out of 320
https://mezcalia.com/producto/mezcal-sertres-espadin-copy/
#152 out of 320
https://mezcalia.com/producto/mezcal-sertres-espadin/
#153 out of 320
https://mezcalia

#247 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-rio-revuelto-ensamble/
#248 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-mezcal-rio-revuelto-ensamble/
#249 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-la-fugitiva/
#250 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-mezcal-la-fugitiva/
#251 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-iba-55/
#252 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-mezcal-iba-55/
#253 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-iba-40-organico/
#254 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-hikma/
#255 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-mezcal-hikma/
#256 out of 320
https://mezcalia.com/producto/regalo-con-mezcal-cesta-grande-mezcal-don-mateo-de-la-sierra-pechuga/
#257 out of 320
https://mezcalia.com/

In [1112]:
#with open("mezcalia_df_raw.pkl", "wb") as fp:  
    #pickle.dump(mezcalia_df, fp)

In [1154]:
with open("mezcalia_df_raw.pkl", "rb") as fp:  
    mezcalia_df = pickle.load(fp)

# Cleaning

#### price

In [1170]:
mezcalia_df['bottle_price'] = mezcalia_df['bottle_price'].str.replace('\.00|\,|\$|[a-zA-Z.]','').str.replace(' \– +','-').str.replace('\d +\d','-').str.replace('   .+','').str.replace(' +','')

#### ABV

In [1229]:
mezcalia_df['ABV_etiq'] = mezcalia_df['etiquetas'].str.lower().str.extract(r'((\d+(?=\% v))|(\d+(?=%,)))')[0]

In [1230]:
mezcalia_df['ABV_descr'] = mezcalia_df['description'].str.lower().str.extract(r'((\d+(?=\% v))|(\d+(?=%,)))')[0]

In [1232]:
mezcalia_df

,website_link,bottle_name,bottle_price,etiquetas,short_description,description,ABV_etiq,ABV_descr
0,https://mezcalia.com/producto/mezcal-gin/,Mezcal Gin,750,"Etiquetas: 40% Vol. Alc., 750 ml., Artesanal, ...",Mezcal Gin Mezcal Gin de agave cenizo y botáni...,Descripción Valoraciones (0) Mezcal Gi...,40,NaN
1,https://mezcalia.com/producto/mezcal-burrito-f...,Mezcal Burrito Fiestero,500,"Etiquetas: 40% Vol. Alc., 700 ml, Agave cenizo...",Mezcal Burrito Fiestero Mezcal Burrito Fiester...,Descripción Valoraciones (0) Mezcal Bu...,40,NaN
2,https://mezcalia.com/producto/senor-maguey-coc...,Señor Maguey Cocktail Miel De Agave,250,"Etiquetas: 100% Organica, 750 ml., Alimentos G...",Señor Maguey Cocktail Miel de Agave Dulzura pu...,Descripción Valoraciones (0) Señor Mag...,NaN,NaN
3,https://mezcalia.com/producto/mezcal-aleron/,Mezcal Alerón,595,"Etiquetas: 38% Vol. Alc., 750 ml., Agave ceniz...",Mezcal Alerón Mezcal Alerón es un Mezcal artes...,Descripción Valoraciones (0) Mezcal Al...,38,NaN
4,https://mezcalia.com/producto/mezcal-ponte-chi...,Mezcal Ponte Chingón Elixir de Agave,626,"Etiquetas: 43% Vol. Alc., 750 ml., Agave Espad...",Mezcal Ponte Chingón Elixir de agave Las produ...,Descripción Información adicional V...,43,NaN
5,https://mezcalia.com/producto/sotol-ono/,Sotol Onó,388,"Etiquetas: 45% Vol. Alc., Artesanal, Dasylirio...",Sotol Onó El sotol es un destilado de la plant...,Descripción Valoraciones (0) Sotol Onó...,45,NaN
6,https://mezcalia.com/producto/mezcal-local-pec...,Mezcal Local Pechuga Box,819,"Etiquetas: 42% Vol. Alc., 50 ml., Artesanal, D...",Mezcal Local Pechuga Box Botella de Mezcal Loc...,Descripción Valoraciones (0) Mezcal Lo...,42,NaN
7,https://mezcalia.com/producto/sotol-flor-del-d...,Sotol Flor Del Desierto Veneno,898,"Etiquetas: Dasylirion Leiophyllum, Sotol, Soto...",Sotol Flor del Desierto Veneno Sotol Flor del ...,Descripción Valoraciones (0) Sotol Flo...,NaN,NaN
8,https://mezcalia.com/producto/mezcal-anejo-joy...,Mezcal Añejo Joya Gran Reserva,134-199,"Etiquetas: 36% vol. Alc, 750 ml., Añejado, Esp...",Mezcal Añejo Joya Gran Reserva La Joya de Oaxa...,Descripción Valoraciones (0) Mezcal Añ...,36,NaN
9,https://mezcalia.com/producto/la-fugitiva-mezc...,Mezcal La Fugitiva Personalizable Edición Espe...,1478,"Etiquetas: 40% Vol. Alc., 750ml, Arroqueño, Ar...",La Fugitiva Mezcal Personalizable Edición Espe...,Descripción Valoraciones (0) La Fugiti...,40,NaN


#### Regalos

In [1243]:
mezcalia_df['etiq+descr'] = mezcalia_df['etiquetas'] + ' ' + mezcalia_df['short_description'] + ' ' +  mezcalia_df['description']

In [1245]:
mezcalia_df['regalos'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'(regal)')

In [1249]:
mezcalia_df[mezcalia_df['regalos'] != 'regal'].isna().sum()

website_link           0
bottle_name           10
bottle_price          10
etiquetas              0
short_description      0
description            0
ABV_etiq              37
ABV_descr            212
etiq+descr             0
regalos              230
dtype: int64

#### Bottle ml

In [1253]:
mezcalia_df['bottle_ml'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'(\d+(?= ml)|\d+(?=ml))')

#### Destillation

In [1264]:
mezcalia_df['destillation'] = mezcalia_df['etiquetas'].str.lower().str.extract(r'(destil.+?(?=\, ))')

#### Style

In [1333]:
pat = '|'.join(mezcalia_meta['style']).lower()
pat = 'abocado|bacanora|crema|ensamble|gusano|licor|orgánico certificado|pechuga|raicilla|sotol'

In [1334]:
mezcalia_df['Style'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'({})'.format(pat))

#### Agave

In [1335]:
pat = '|'.join(mezcalia_meta['agave']).lower()
pat

'americano serrano|cenizo|coyote|cupreata-papalote|espadin|lechugilla|jabalí|mexicano|madrecuixe. bicuishe. barril|salmiana|sierra negra|tepextate|tobala. papalometl|otros agaves'

In [1336]:
mezcalia_df['agave'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'({})'.format(pat))

#### Aged

In [1319]:
pat = '|'.join(mezcalia_meta['aged']).lower()
pat

'joven|reposado|añejo|otros'

In [1321]:
mezcalia_df['aged'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'({})'.format(pat))

#### Region

In [1322]:
pat = '|'.join(mezcalia_meta['region']).lower()
pat

'durango|guanajuato|guerrero|michoacán|oaxaca|puebla|san luis potosí|tamaulipas|zacatecas|otras'

In [1324]:
mezcalia_df['region'] = mezcalia_df['etiq+descr'].str.lower().str.extract(r'({})'.format(pat))

#### Not Mezcal

In [1328]:
mezcalia_df['gourmet_cata'] = mezcalia_df['etiquetas'].str.lower().str.extract(r'(gourmet|cata)')

In [1331]:
mezcalia_df[mezcalia_df['gourmet_cata'].isna()].shape[0]/mezcalia_df.shape[0]

0.9688473520249221

### Getting Images

In [1383]:
mezcalia_df_clean = mezcalia_df[(mezcalia_df['gourmet_cata'].isna()) & (mezcalia_df['regalos'].isna())].reset_index(drop=True)
mezcalia_df_clean.shape

(221, 17)

In [1384]:
mezcalia_df_clean['images'] = np.nan

In [1386]:
scraper = cfscrape.create_scraper()
for i in range(80, mezcalia_df_clean.shape[0]):
    url = mezcalia_df_clean['website_link'].loc[i]
    
    print('#{} out of 221'.format(i))
    print(url)
    sleep(randint(20,30))
    def get_page_meta():
        scraped_html=scraper.get(url).content
        html = etree.HTML(scraped_html)
        image = html.xpath('//*[@class="ast-woocommerce-container"]//div//a[contains(@href, "jpg")]/@href')
        print(image)
        mezcalia_df_clean['images'].loc[i] = image
    try:
        func_timeout(60, get_page_meta)
            
    except FunctionTimedOut:
        print('Timeout')
        continue

#80 out of 221
https://mezcalia.com/producto/mezcal-naroba-tepextate/
['https://mezcalia.com/wp-content/uploads/2019/05/naroba-espadin-1.jpg']
#81 out of 221
https://mezcalia.com/producto/mezcal-rey-zapoteco-joven-espadin-copy-copy-2/
['https://mezcalia.com/wp-content/uploads/2019/03/PECHUGA-NAC.jpg']
#82 out of 221
https://mezcalia.com/producto/sal-de-aqui-gusano-de-maguey/
['https://mezcalia.com/wp-content/uploads/2019/02/sda-gusano-50g.jpg']
#83 out of 221
https://mezcalia.com/producto/mezcal-barro-de-cobre-ensamble/
['https://mezcalia.com/wp-content/uploads/2019/01/bdc-ens.jpg']
#84 out of 221
https://mezcalia.com/producto/mezcal-los-cuerudos-reposado/
['https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_7387655433933756780-1.jpg']
#85 out of 221
https://mezcalia.com/producto/mezcal-los-cuerudos-tobala/
['https://mezcalia.com/wp-content/uploads/2019/01/Los-Cuerudos-Tobalá-7x12.jpg']
#86 out of 221
https://mezcalia.com/producto/mezcal-el-viejo-manuelon-anejo-do

['https://mezcalia.com/wp-content/uploads/2019/10/cuish-tobalá.jpg']
#131 out of 221
https://mezcalia.com/producto/mezcal-lalocura-tobasiche/
['https://mezcalia.com/wp-content/uploads/2019/10/lalocura-tobasiche-1.jpg']
#132 out of 221
https://mezcalia.com/producto/mezcal-memorable-madrecuishe/
['https://mezcalia.com/wp-content/uploads/2019/09/mezcal-memorable-madrecuishe-e1577939117733.jpg']
#133 out of 221
https://mezcalia.com/producto/mezcal-memorable-tepextate/
['https://mezcalia.com/wp-content/uploads/2019/11/memorable_tepeztate.jpg']
#134 out of 221
https://mezcalia.com/producto/mezcal-memorable-bicuishe/
['https://mezcalia.com/wp-content/uploads/2019/09/memorable_bicuishe.jpg']
#135 out of 221
https://mezcalia.com/producto/mezcal-memorable-arroqueno/
['https://mezcalia.com/wp-content/uploads/2019/09/memorable-arroqueno.jpg']
#136 out of 221
https://mezcalia.com/producto/mezcal-memorable-tobasiche/
['https://mezcalia.com/wp-content/uploads/2019/11/memorable-tobasiche.jpg']
#137 ou

['https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_14742871783994112486-1.jpg', 'https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_7165695487594419358-6.jpg']
#180 out of 221
https://mezcalia.com/producto/mezcal-madrona-joven/
['https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_14742871376073070903-1.jpg']
#181 out of 221
https://mezcalia.com/producto/mezcal-barro-de-cobre-espadin/
['https://mezcalia.com/wp-content/uploads/2019/01/bdc-esp.jpg']
#182 out of 221
https://mezcalia.com/producto/mezcal-amaxak/
['https://mezcalia.com/wp-content/uploads/2019/01/amaxak-espadín-joven-38.jpg']
#183 out of 221
https://mezcalia.com/producto/pox-tres-almas/
['https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_14833393499850922037-1.jpg']
#184 out of 221
https://mezcalia.com/producto/mezcal-altares-de-mi-tierra-madrecuishe/
['https://mezcalia.com/wp-content/uploads/2019/05/5cbfa74e4eca2c00077daf9e_109382905685

In [9]:
mezcalia_df_clean.head(1)

,website_link,bottle_name,bottle_price,etiquetas,short_description,description,ABV_etiq,ABV_descr,etiq+descr,regalos,bottle_ml,destillation,Style,agave,aged,region,gourmet_cata,images,main_img
0,https://mezcalia.com/producto/mezcal-gin/,Mezcal Gin,750,"Etiquetas: 40% Vol. Alc., 750 ml., Artesanal, ...",Mezcal Gin Mezcal Gin de agave cenizo y botáni...,Descripción Valoraciones (0) Mezcal Gi...,40,NaN,"Etiquetas: 40% Vol. Alc., 750 ml., Artesanal, ...",NaN,750,destilado de metal,pechuga,cenizo,otros,durango,NaN,[https://mezcalia.com/wp-content/uploads/2017/...,https://mezcalia.com/wp-content/uploads/2017/0...


## Clean ABV

In [21]:
mezcalia_df_clean['ABV_etiq'] = mezcalia_df_clean['ABV_etiq'].astype(float)
import math
for i in range(mezcalia_df_clean.shape[0]):
    if math.isnan(mezcalia_df_clean['ABV_etiq'].loc[i]):
        mezcalia_df_clean['ABV_etiq'].loc[i] = mezcalia_df_clean['ABV_descr'].loc[i]

### Main img

In [8]:
mezcalia_df_clean['main_img'] = mezcalia_df_clean['images'].apply(lambda x: x[0] if len(x)>0 else x)

In [23]:
mezcalia_df_clean.head(1)

,website_link,bottle_name,bottle_price,etiquetas,short_description,description,ABV_etiq,ABV_descr,etiq+descr,regalos,bottle_ml,destillation,Style,agave,aged,region,gourmet_cata,images,main_img
0,https://mezcalia.com/producto/mezcal-gin/,Mezcal Gin,750,"Etiquetas: 40% Vol. Alc., 750 ml., Artesanal, ...",Mezcal Gin Mezcal Gin de agave cenizo y botáni...,Descripción Valoraciones (0) Mezcal Gi...,40,NaN,"Etiquetas: 40% Vol. Alc., 750 ml., Artesanal, ...",NaN,750,destilado de metal,pechuga,cenizo,otros,durango,NaN,[https://mezcalia.com/wp-content/uploads/2017/...,https://mezcalia.com/wp-content/uploads/2017/0...


In [62]:
with open("mezcalia_df_processed.pkl", "wb") as fp:  
     pickle.dump(mezcalia_df_clean, fp)

## To Google Sheets

In [37]:
cols = ['bottle_name','bottle_price','bottle_ml','ABV_etiq','destillation', 'Style','aged','agave','region',
        'short_description','description','main_img','images','website_link']

In [38]:
new_cols = {}
for col in cols:
    new_col = re.sub('_', ' ', col)
    new_col = new_col[0].upper()+new_col[1:]
    new_cols[col] = new_col

In [40]:
to_sheets = mezcalia_df_clean[cols].fillna('').rename(columns=new_cols)

In [41]:
wks_name = 'Mezcalia'
cell_of_start_df = 'A1'
d2g.upload(to_sheets,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=False)

<Worksheet 'Mezcalia' id:909611884>

# Totalwine

#### getting links

In [1418]:
links = []
for i in range(1,5):
    sleep(randint(10,15))
    
    url = 'https://www.totalwine.com/spirits/mezcal/c/000889?viewall=true&page={}&pageSize=24'.format(i)
    scraped_html=scraper.get(url).content
    html = etree.HTML(scraped_html)
    
    page_links_raw = html.xpath('//*[@class="grid__1eZnNfL-"]//@href')
    for link in page_links_raw:
        if link not in links and 'tab' not in link:
            links.append(link)

In [1419]:
len(links)

76

In [1423]:
links = ['https://www.totalwine.com' + x for x in links]

#### getting metadata

In [1585]:
path = '/Users/macbook/Downloads/Proxies - Sheet1.csv'
proxies = pd.read_csv(path)
proxies.columns = ['proxies']
proxy_pool = cycle(proxies['proxies'].values[-40:])

In [1612]:
login = 'usr14362974'
password = 'ps143621582973974'

In [1582]:
total_wine = pd.DataFrame([])

In [1591]:
for i in range(len(links)):
    url = links[i]
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}

    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    
    print('#{} out of 76'.format(i))
    print(url)
    sleep(randint(1,2))
    #def get_page_meta():
    scraped_html=scraper.get(url).content
    html = etree.HTML(scraped_html)
    name = html.xpath('//*[@class="productTitle__3XDd9UVh"]//text()')[0]
    bot_ml = html.xpath('//*[@class="productSubTitle__3sTXP-XD"]//text()')[0]
    price = html.xpath('//*[@id="edlpPrice"]//text()')[0]
    img = html.xpath('//*[@class="prodImage__aq-qLP3Q"]//@src')[0]
    meta_names = html.xpath('//*[@class="detailsTableLabel__kj9FI2Mq"]//text()')
    metadata = html.xpath('//*[@class="detailsTableText__1SvcRdYn"]//text()')
    page_data = pd.DataFrame({'Bottle Name':name,
                              'Bottle ml':bot_ml,
                              'Price_TW':price,
                              'Image_TW':img,
                              'Bottle Link_TW':url}, index=[0])

    page_meta = pd.DataFrame([metadata], columns = meta_names)
    page = pd.concat([page_data, page_meta], axis=1)
    total_wine = pd.concat([total_wine, page], axis=0)

#0 out of 76
https://www.totalwine.com/spirits/mezcal/casamigos-mezcal-joven/p/192858750?s=1108&igrules=true
#1 out of 76
https://www.totalwine.com/spirits/mezcal/ilegal-mezcal-joven/p/113411750?s=1108&igrules=true
#2 out of 76
https://www.totalwine.com/spirits/mezcal/xicaru-mezcal-silver/p/171655750?s=1108&igrules=true
#3 out of 76
https://www.totalwine.com/spirits/mezcal/luminar-mezcal-joven/p/219093750?s=1108&igrules=true
#4 out of 76
https://www.totalwine.com/spirits/mezcal/luminar-mezcal-anejo/p/219095750?s=1108&igrules=true
#5 out of 76
https://www.totalwine.com/spirits/mezcal/monte-alban-mezcal/p/662750?s=1108&igrules=true
#6 out of 76
https://www.totalwine.com/spirits/mezcal/yuu-baal-mezcal-joven/p/172314750?s=1108&igrules=true
#7 out of 76
https://www.totalwine.com/spirits/mezcal/casamigos-mezcal-joven/p/217049375?s=1108&igrules=true
#8 out of 76
https://www.totalwine.com/spirits/mezcal/del-maguey-san-luis-del-rio/p/119159750?s=1108&igrules=true
#9 out of 76
https://www.totalw

#73 out of 76
https://www.totalwine.com/spirits/mezcal/wild-shot-silver-mezcal/p/117106750?s=1108&igrules=true
#74 out of 76
https://www.totalwine.com/spirits/mezcal/del-maguey-san-luis-del-rio-azul/p/218363750?s=1108&igrules=true
#75 out of 76
https://www.totalwine.com/spirits/mezcal/mezcal-tosba-espadin/p/183648750?s=1108&igrules=true


In [1604]:
cols = ['Bottle Name', 'BRAND', 'Bottle ml', 'Price_TW','Bottle Link_TW','Image_TW', 'TASTE']

In [1606]:
total_wine = total_wine[cols].iloc[76:].reset_index(drop=True)
total_wine = total_wine.rename(columns={'Bottle ml':'Bottle ml_TW','BRAND':'Brand_TW','TASTE':'Taste_TW'})
total_wine['origin'] = 'total_wine'

In [1732]:
total_wine.shape#head()

(76, 8)

In [1733]:
with open("total_wine_no_rev.pkl", "wb") as fp:  
    pickle.dump(total_wine, fp)

# Missionliquor

In [1653]:
## Selenium
page_links = []
browser = webdriver.Chrome('/Users/macbook/Downloads/chromedriver')

In [1678]:
url = 'https://www.missionliquor.com/collections/spirits?page=6&_=pf&pf_pt_product_type=MEZCAL'
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
for anchor in soup.find_all("a"):
    if "href" in anchor.attrs and 'collections/spirits/products' in anchor.attrs['href']:
        page_links.append(anchor.attrs['href'])

In [1680]:
len(page_links)

283

In [1684]:
page_links = ['https://www.missionliquor.com'+x for x in page_links]

#### page data

In [ ]:
proxy = next(proxy_pool)
go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
s = cfscrape.create_scraper()
s.proxies = go_proxy
scraper = cfscrape.create_scraper(sess=s)

In [1699]:
missionliquor = pd.DataFrame({'Bottle Name':np.nan,
                              'origin':'missionliquor',
                              'Price_ML':np.nan,
                              'Bottle Link_ML':page_links,
                              'Image_ML':np.nan})

In [ ]:
for i in range(missionliquor.shape[0]):
    print('#{} out of {}'.format(i, missionliquor.shape[0]))
    url = missionliquor['Bottle Link_ML'].loc[i]
    proxy = next(proxy_pool)
    go_proxy = { 'https' : 'https://{}:{}@{}'.format(login, password, proxy)}
    s = cfscrape.create_scraper()
    s.proxies = go_proxy
    scraper = cfscrape.create_scraper(sess=s)
    scraped_html=scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    
    name = html.xpath('//*[@class="product-info__title"]/text()')
    if name:
        missionliquor['Bottle Name'].loc[i] = name[0]
    
    price = html.xpath('//*[@class="product-price"]/text()')
    if price:
        price = re.sub('\n| +','',price[0])
        missionliquor['Price_ML'].loc[i] = price
    
    image = html.xpath('//*[@class="product-featured"]/@src')
    if image:
        missionliquor['Image_ML'].loc[i] = image[0]

In [1722]:
missionliquor['Image_ML'] = missionliquor['Image_ML'].apply(lambda x: 'https:'+x)

In [1725]:
with open("missionliquor_no_rev.pkl", "wb") as fp:  
    pickle.dump(missionliquor, fp)

# la europea

In [3077]:
browser = webdriver.Chrome('/Users/macbook/Downloads/chromedriver')

In [3121]:
url = 'https://www.laeuropea.com.mx/productos/categoria/licores/subcategoria/mezcales'
browser.get(url)

In [3137]:
laeuropea = pd.DataFrame([])

In [ ]:
for i in range(1, 99): #99
    print('#',i)
    url = 'https://www.laeuropea.com.mx/productos/categoria/licores/subcategoria/mezcales'
    browser.get(url)
    sleep(randint(2,3))
    
    # press 'more' button 
    if i >=19: 
        path = '/html/body/app-root/div/app-subcategory/div/div[2]/div[4]/button'
        browser.find_element_by_xpath(path).click()
        sleep(randint(1,3))
        if i >=39:
            browser.find_element_by_xpath(path).click()
            sleep(randint(1,3))
            if i >=59:
                browser.find_element_by_xpath(path).click()
                sleep(randint(1,3))
                if i >=79:
                    browser.find_element_by_xpath(path).click()
                    sleep(randint(1,3))
            
    #browser.explicitly_wait(5)   
    
    path = '/html/body/app-root/div/app-subcategory/div/div[2]/div[3]/div[{}]/le-general-card/div/div[3]/div[2]/a[2]'.format(i)
    element = browser.find_element_by_xpath(path)
    browser.execute_script("arguments[0].click();", element)

    sleep(randint(2,4))
    #d
    try:
        # bottle name
        path = '//*[@class="le-text-18 le-tw-bold le-card-details-name"]' 
        name = browser.find_element_by_xpath(path).text
    except:
        name = np.nan
    try:
        # bottle ABV, Volume
        path = '//*[@class="le-card-details-charact le-text-center"]' 
        abv_volume = browser.find_element_by_xpath(path).text
    except:
        abv_volume = np.nan
    
    try:   
        # main metadata 
        path = '//*[@class="le-card-details-factors"]' 
        main_metadata  = browser.find_element_by_xpath(path).text
    except:
        main_metadata = np.nan
    
    try:
        # region
        path = '//*[@class="le-card-details-charact le-text-center ng-star-inserted"]' 
        region = browser.find_element_by_xpath(path).text
    except:
        region = np.nan
    
    try:
        # price
        path = '//*[@class="le-card-details-price le-text-16 le-text-center"]' 
        price = browser.find_element_by_xpath(path).text
    except:
        price = np.nan
    
    try:
        # image
        path = '//*[@id="carouselDetails"]/div/div[1]/ngx-image-zoom/div/div/img' 
        img = browser.find_element_by_xpath(path).get_attribute("src")
    except:
        img = np.nan
    
    link = browser.current_url
    row = pd.DataFrame({'Bottle Name':name,
                        'origin':'laeuropea',
                        'ABV_Volume_LE':abv_volume,
                        'Price_LE':price,
                        'Main_metadata_LE':main_metadata,
                        'Region_LE':region,
                        'Bottle Link_LE':link,
                        'Image_LE':img}, index=[i])
    laeuropea = pd.concat([laeuropea, row], axis=0)

In [3139]:
with open("laeuropea_no_rev.pkl", "wb") as fp:  
    pickle.dump(laeuropea, fp)

## cleaning

In [3184]:
meta_table = pd.DataFrame([])
for idx in laeuropea.index:
    data = laeuropea['Main_metadata_LE'][idx].split('\n')[1::2]
    columns = laeuropea['Main_metadata_LE'][idx].split('\n')[::2]
    row = pd.DataFrame([data], columns = columns, index=[idx])
    meta_table = pd.concat([meta_table, row], axis=0)
laeuropea = pd.concat([laeuropea, meta_table], axis=1)

In [3191]:
laeuropea['Bottle Name'] = laeuropea['Bottle Name'].str.replace(r'( \- \d.+| \-.+|\-.+| \d\d\d ML| \d\d\d ml)','')
laeuropea['Bottle_Volume_LE'] = laeuropea['ABV_Volume_LE'].str.extract(r'(\d+(?=ml))')
laeuropea['ABV_LE'] = laeuropea['ABV_Volume_LE'].str.extract(r'(\d+(?=\%))')

laeuropea['Price_LE'] = laeuropea['Price_LE'].str.extract(r'((?<=\$)\d.+(?= \$)|(?<=\$)\d.+(?= ))')
laeuropea['Price_LE'] = laeuropea['Price_LE'].str.replace(',','').astype(float)
laeuropea['Region_LE'] = laeuropea['Region_LE'].str.extract(r'((?<=Región:).+(?= Varietal))')
laeuropea['Agave_LE'] = laeuropea['Region_LE'].str.extract(r'((?<=Varietal:).+(?=\n))')

laeuropea = laeuropea.replace('No aplica', np.nan, regex=True)
laeuropea.drop(columns = ['ABV_Volume_LE', 'Main_metadata_LE'], inplace=True)
laeuropea = laeuropea.rename(columns = {'Vista':'View', 'Olor':'Odor','Sabor':'Taste','Meridaje':'Pairing'})

In [3210]:
with open("laeuropea_no_rev.pkl", "wb") as fp:  
    pickle.dump(laeuropea, fp)

# Bodega

In [3501]:
browser = webdriver.Chrome('/Users/macbook/Downloads/chromedriver')

In [3502]:
url = 'https://www.bodegasalianza.com/mezcal?PS=12&O=OrderByTopSaleDESC'
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [3513]:
links = []
for anchor in soup.find_all("a"):
    if "href" in anchor.attrs and 'mezcal' in anchor.attrs['href'] and '/p' in anchor.attrs['href'] and anchor.attrs['href'] not in links:
        links.append(anchor.attrs['href'])

In [3516]:
s = cfscrape.create_scraper()
scraper = cfscrape.create_scraper(sess=s)

In [3635]:
bodega = pd.DataFrame(links, columns=['Bottle Link_BD'])

In [3638]:
bodega['origin'] = 'BD'
bodega['Bottle Name'] = np.nan
bodega['Price_BD'] = np.nan
bodega['Image_BD'] = np.nan
bodega['Description'] = np.nan

In [3650]:
for i in range(bodega.shape[0]):
    if i%10==0:
        print('#',i)
    sleep(randint(10,20))
    
    url = bodega['Bottle Link_BD'].loc[i]
    scraped_html = scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    
    name = html.xpath('/html/body/div/div/div/div[2]/div[2]/div/div/h1/div//text()')
    if name:
        bodega['Bottle Name'].loc[i] = name[0]
        
    price = html.xpath('//*[@class="skuBestPrice"]//text()')
    if price:
        bodega['Price_BD'].loc[i] = price[0]
    
    image = html.xpath('/html/body/div/div/div/div/div/div/div//img//@src')
    if image:
        bodega['Image_BD'].loc[i] = image[0]
    descr = html.xpath('//*[@class="productDescription"]//text()')
    if descr:
        bodega['Description'].loc[i] = ', '.join(descr)

# 0
# 10
# 20
# 30
# 40
# 50
# 60
# 70
# 80
# 90
# 100


## Cleaning
#### Names

In [3675]:
bodega = bodega[~bodega['Bottle Name'].fillna('').str.contains('C/Bot|\+|C/Ensamble')].reset_index(drop=True)
bodega.shape

(99, 6)

In [3683]:
bodega['Bottle Volume_BD'] = bodega['Bottle Name'].str.lower().str.extract(r'(\d+(?=ml)|\d+(?= ml))')

In [3710]:
bodega['Bottle Name'] =  bodega['Bottle Name'].str.replace(r'(\d+ ml|\d+ml|\d\.\d+L| \-.+|100%|C\/.+|\(\d\d°\)| +$)','')
bodega['Bottle Name'] = bodega['Bottle Name'].str.replace(r'(  +)',' ').str.replace(r'( +$)','')

In [3700]:
print(bodega['Bottle Link_BD'].loc[49])

https://www.bodegasalianza.com/mezcal-verde-momento-surreal-700ml-31798/p


#### Price

In [3715]:
bodega['Price_BD'] = bodega['Price_BD'].str.replace(r'(\$|\,)','').astype(float)

#### ABV

In [3726]:
bodega['ABV_BD'] = bodega['Description'].str.extract(r'((?<=Vol).+(?=%))')
bodega['ABV_BD'] = bodega['ABV_BD'].str.extract(r'(\d+\.\d+|\d+)')

In [3728]:
with open("bodega_no_rev.pkl", "wb") as fp:  
    pickle.dump(bodega, fp) 

# Mercadodemezcal

In [3737]:
url = 'https://ca.mercadodemezcal.com/mezcal/?sort=featured&page={}'.format(21)
scraped_html = scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)

In [ ]:
links = []
for i in range(1,25):
    print('#',i)
    sleep(randint(10,20))
    url = 'https://ca.mercadodemezcal.com/mezcal/?sort=featured&page={}'.format(i)
    scraped_html = scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)

    for link in html.xpath('//*[@id="product-listing-container"]//a/@href'):
        if 'sort' not in link:
            links.append(link)

In [3744]:
mercadode = pd.DataFrame(links, columns = ['Bottle_Link_MD'])

In [3770]:
mercadode['origin'] = 'MD'
mercadode['Bottle Name'] = np.nan
mercadode['Price_MD'] = np.nan
mercadode['Brand_MD'] = np.nan
mercadode['Image_MD'] = np.nan
mercadode['Description'] = np.nan

In [3785]:
for i in range(mercadode.shape[0]):
    if i%10==0:
        print('#',i)
    
    sleep(randint(5,15))
    url = mercadode['Bottle_Link_MD'].loc[i]
    scraped_html = scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    
    name = html.xpath('//*[@class="productView-title"]//text()')
    if name:
        mercadode['Bottle Name'].loc[i] = name[0]
    
    brand = html.xpath('//*[@class="productView-brand"]//text()')
    if brand:
        brand = ''.join(brand)
        brand = re.sub('\n|  +', '', brand)
        mercadode['Brand_MD'].loc[i] = brand
    
    descr = html.xpath('//*[@class="tab-content is-active"]//text()')
    
    if descr:
        descr = ''.join(descr)
        descr = re.sub('\n|  +', '', descr)
        mercadode['Description'].loc[i] = descr
    
    img = html.xpath('//*[@class="productView-img-container"]//@href')
    if img:
        mercadode['Image_MD'].loc[i] = img[0]
    
    price = html.xpath('/html/body/div[2]/div[1]/div/div[1]/section[1]/div/div[1]/div[3]/span[3]//text()')
    if price:
        mercadode['Price_MD'].loc[i] = price[0]

# 0
# 10
# 20
# 30
# 40
# 50
# 60
# 70
# 80
# 90
# 100
# 110
# 120
# 130
# 140
# 150
# 160
# 170
# 180
# 190
# 200
# 210
# 220
# 230
# 240
# 250
# 260
# 270
# 280
# 290


## Cleaning
#### Names

In [3793]:
mercadode['Bottle Volume_MD'] = mercadode['Bottle Name'].str.extract(r'((?<=\().+(?=ml\)))')

In [3861]:
mercadode.rename(columns={'Bottle Volume_BD':'Bottle Volume_MD'}, inplace=True)

In [3818]:
mercadode['Bottle Name'] = mercadode['Bottle Name'].str.replace(r'(\(\d.+\)|Alc/Vol|\d+%|\d+\.\d+%)','')
mercadode['Bottle Name'] = mercadode['Bottle Name'].str.replace(r'( +$)','')

In [3856]:
mercadode['Bottle Name'] = mercadode['Bottle Name'].str.replace(r'(Espa.+n)','Espadin').str.replace(r'(AÃ±ejo)','Anejo')
mercadode['Bottle Name'] = mercadode['Bottle Name'].str.replace(r'(EspÃ­ritu)','Espiritu')

#### Price

In [3865]:
mercadode['Price_MD'] = mercadode['Price_MD'].str.replace(r'(\$|\,)','').astype(float)

In [3871]:
with open("mercadode_no_rev.pkl", "wb") as fp:  
    pickle.dump(mercadode, fp)

# Preparing to GS

### Reading all tables

In [314]:
with open("rev_table_processed.pkl", "rb") as fp:  
    mezcal_rev = pickle.load(fp)

In [315]:
rename_cols = {}    
for col in mezcal_rev.columns:
    if col != 'origin' and col!='Bottle Name':
        rename_cols[col] = col+'_MR' 

In [316]:
mezcal_rev = mezcal_rev.rename(columns=rename_cols)
mezcal_rev = mezcal_rev.rename(columns={'Cost_MR':'Price_MR','Distillation_MR':'Destillation_MR','Review link_MR':'Bottle_link_MR'})

In [317]:
cols = ['Bottle Name', 'Rating_MR', 'Price_MR', 'Brand_MR', 'NOM_MR', 'Mezcalero_MR',
       'Agave_MR', 'Grind_MR', 'Destillation_MR', 'Style_MR', 'State_MR',
       'Town_MR', 'ABV_MR', 'Website_MR', 'Image Link_MR', 'Bottle_link_MR']

In [318]:
mezcal_rev = mezcal_rev.replace(r'(, , , | , , ,)',', ', regex=True)

In [319]:
mezcal_rev = mezcal_rev[cols]
mezcal_rev.head(1)

,Bottle Name,Rating_MR,Price_MR,Brand_MR,NOM_MR,Mezcalero_MR,Agave_MR,Grind_MR,Destillation_MR,Style_MR,State_MR,Town_MR,ABV_MR,Website_MR,Image Link_MR,Bottle_link_MR
0,Mitre Origen,3,4.0,Mitre Mezcal,NOM-O338X,Don Tacho,Espadin,Tahona,Copper,Joven,Oaxaca,Santiago Matatlan,37.5-40%,https://www.facebook.com/mezcalmitre/,https://www.mezcalreviews.com/wp-content/uploa...,https://www.mezcalreviews.com/mezcal/mitre-joven/


### old_tequila

In [134]:
with open("old_tequila_processed.pkl", "rb") as fp:  
       old_tequila = pickle.load(fp)

In [137]:
old_tequila.head(1)

,Bottle Link_OT,Bottle Name,Brand_OT,Price_OT,Rating_OT,bot_description,bot_num_reviews,Image_OT,rev_body_1,rev_rating_1,rev_author_date_1,rev_body_2,rev_rating_2,rev_author_date_2,rev_body_3,rev_rating_3,rev_author_date_3,rev_body_4,rev_rating_4,rev_author_date_4,rev_body_5,rev_rating_5,rev_author_date_5,rev_body_6,rev_rating_6,rev_author_date_6,rev_body_7,rev_rating_7,rev_author_date_7,rev_body_8,rev_rating_8,rev_author_date_8,rev_body_9,rev_rating_9,rev_author_date_9
0,https://www.oldtowntequila.com/400-conejos-esp...,400 Conejos Espadin Mezcal Artesanal,400 Conejos,$27.99,3,"[\n , 400 Conejos Espad...",1,https://cdn11.bigcommerce.com/s-u9ww3di/images...,400 Conejos Mezcal. Has a little bite well pri...,3.0,Posted by Thomas on May 2nd 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
old_tequila = old_tequila.rename(columns={'bot_link':'Bottle Link_OT', 
                                          'bot_brand':'Brand_OT',
                                          'bot_price':'Price_OT',
                                          'bot_rating':'Rating_OT',
                                          'bot_name':'Bottle Name',
                                         'bot_num_reviews':'# Reviews'})

In [154]:
#for i in old_tequila.index:
    #print(i, old_tequila['Description'].loc[i])

In [158]:
old_tequila['Description'] = np.nan
for i in old_tequila.index:
    descr = old_tequila['bot_description'].loc[i]
    if type(descr)==list and len(descr)>0:
        descr = ' '.join(descr)
        descr = re.sub('\n+|\r+', '', descr)
        descr = re.sub('  +', ' ', descr)
        descr = re.sub('^ | $|', '', descr)
        descr = re.sub(' ,,,|,,, ', '', descr)
        descr = re.sub(r'(\xa0)', '', descr)
        
        old_tequila['Description'].loc[i] = descr
        
cols = ['Bottle Name', 'Description','Brand_OT', 'Price_OT', 'Rating_OT','Bottle Link_OT','# Reviews', 'Image_OT',
       'rev_body_1', 'rev_rating_1', 'rev_author_date_1', 'rev_body_2',
       'rev_rating_2', 'rev_author_date_2', 'rev_body_3', 'rev_rating_3',
       'rev_author_date_3', 'rev_body_4', 'rev_rating_4', 'rev_author_date_4',
       'rev_body_5', 'rev_rating_5', 'rev_author_date_5', 'rev_body_6',
       'rev_rating_6', 'rev_author_date_6', 'rev_body_7', 'rev_rating_7',
       'rev_author_date_7', 'rev_body_8', 'rev_rating_8', 'rev_author_date_8',
       'rev_body_9', 'rev_rating_9', 'rev_author_date_9']

In [160]:
old_tequila = old_tequila[cols]
old_tequila.head(1)

,Bottle Name,Description,Brand_OT,Price_OT,Rating_OT,Bottle Link_OT,# Reviews,Image_OT,rev_body_1,rev_rating_1,rev_author_date_1,rev_body_2,rev_rating_2,rev_author_date_2,rev_body_3,rev_rating_3,rev_author_date_3,rev_body_4,rev_rating_4,rev_author_date_4,rev_body_5,rev_rating_5,rev_author_date_5,rev_body_6,rev_rating_6,rev_author_date_6,rev_body_7,rev_rating_7,rev_author_date_7,rev_body_8,rev_rating_8,rev_author_date_8,rev_body_9,rev_rating_9,rev_author_date_9
0,400 Conejos Espadin Mezcal Artesanal,400 Conejos Espadin Mezcal Artesanal Joven 750...,400 Conejos,$27.99,3,https://www.oldtowntequila.com/400-conejos-esp...,1,https://cdn11.bigcommerce.com/s-u9ww3di/images...,400 Conejos Mezcal. Has a little bite well pri...,3.0,Posted by Thomas on May 2nd 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### mezcalia

In [324]:
with open("mezcalia_df_processed.pkl", "rb") as fp:  
    mezcalia = pickle.load(fp)

In [325]:
mezcalia = mezcalia.rename(columns={'website_link':'Bottle Link_MZ', 
                                    'bottle_price':'Price_MZ',
                                    'agave':'Agave_MZ',
                                    'ABV_etiq':'ABV_MZ',
                                    'bottle_name':'Bottle_Name',
                                    'destillation':'Destillation_MZ',
                                    'aged':'Style_MZ',
                                    'agave':'Agave_MZ',
                                    'region':'State_MZ',
                                    'images':'Images',
                                    'main_img':'Main Image_MZ',
                                    'short_description':'Short Description',
                                    'description':'Description',
                                    'bottle_ml':'Bottle ml',
                                    'bottle_name':'Bottle Name'
                                   })
mezcalia.drop(columns=['etiquetas','ABV_descr','etiq+descr','regalos','gourmet_cata'], inplace=True)

In [326]:
cols = ['Bottle Name', 'Price_MZ','Short Description','Description',   
        'ABV_MZ', 'Bottle ml', 'Destillation_MZ', 'Style',
        'Agave_MZ', 'Style_MZ', 'State_MZ', 'Bottle Link_MZ','Main Image_MZ','Images']

In [327]:
mezcalia = mezcalia[cols]
mezcalia.head(1)

,Bottle Name,Price_MZ,Short Description,Description,ABV_MZ,Bottle ml,Destillation_MZ,Style,Agave_MZ,Style_MZ,State_MZ,Bottle Link_MZ,Main Image_MZ,Images
0,Mezcal Gin,750,Mezcal Gin Mezcal Gin de agave cenizo y botáni...,Descripción Valoraciones (0) Mezcal Gi...,40,750,destilado de metal,pechuga,cenizo,otros,durango,https://mezcalia.com/producto/mezcal-gin/,https://mezcalia.com/wp-content/uploads/2017/0...,[https://mezcalia.com/wp-content/uploads/2017/...


### Drizly

In [328]:
with open("drizly_data_raw.pkl", "rb") as fp:  
    drizly = pickle.load(fp)

In [329]:
drizly = drizly.rename(columns={'links':'Bottle Link_DZ', 
                                'price-one':'Price_DZ',
                                'price-one-bottle':'Bottle ml_DZ',
                                'price-two':'Price-two_DZ',
                                'price-two-bottle':'Bottle-two ml_DZ',
                                'Town':'Town_DZ',
                                'rateing':'Rating_DZ',
                                'ABV':'ABV_DZ',
                                'Town':'Town_DZ',
                                'rev_num':'# Reviews',
                                'description':'Description',
                                'name':'Bottle Name',
                                'image':'Image_DZ'})
drizly.drop(columns=['metadata','price','rate_review','abv_town','rev_page_count',
                     '1_rev_stars', '1_rev_text', '2_rev_stars', '2_rev_text', '3_rev_stars',
                     '3_rev_text', '4_rev_stars', '4_rev_text', '5_rev_stars', '5_rev_text',
                     '6_rev_stars', '6_rev_text', '7_rev_stars', '7_rev_text', '8_rev_stars',
                     '8_rev_text', '9_rev_stars', '9_rev_text', '10_rev_stars',
                     '10_rev_text','rating','given_rev_num'], inplace=True)

In [330]:
sorted_cols = []
for i in range(1,11):
    for col in drizly.columns:
        if str(i)+'_rev' in col and 'rev_text' not in col:
            sorted_cols.append(col)

In [331]:
cols = ['Bottle Name', 'Description','Price_DZ', 'Bottle ml_DZ','Price-two_DZ', 'Bottle-two ml_DZ', 'Rating_DZ',
        'ABV_DZ', 'Town_DZ', 'Bottle Link_DZ', 'Image_DZ', '# Reviews']
cols.extend(sorted_cols)

In [332]:
drizly = drizly[cols]
drizly.head(1)

,Bottle Name,Description,Price_DZ,Bottle ml_DZ,Price-two_DZ,Bottle-two ml_DZ,Rating_DZ,ABV_DZ,Town_DZ,Bottle Link_DZ,Image_DZ,# Reviews,1_rev_Date,1_rev_Reviewer Name,1_rev_Text,1_rev_Verified Buyer,2_rev_Date,2_rev_Reviewer Name,2_rev_Text,2_rev_Verified Buyer,3_rev_Date,3_rev_Reviewer Name,3_rev_Text,3_rev_Verified Buyer,4_rev_Date,4_rev_Reviewer Name,4_rev_Text,4_rev_Verified Buyer,5_rev_Date,5_rev_Reviewer Name,5_rev_Text,5_rev_Verified Buyer,6_rev_Date,6_rev_Reviewer Name,6_rev_Text,6_rev_Verified Buyer,7_rev_Date,7_rev_Reviewer Name,7_rev_Text,7_rev_Verified Buyer,8_rev_Date,8_rev_Reviewer Name,8_rev_Text,8_rev_Verified Buyer,9_rev_Date,9_rev_Reviewer Name,9_rev_Text,9_rev_Verified Buyer,10_rev_Date,10_rev_Reviewer Name,10_rev_Text,10_rev_Verified Buyer
0,Del Maguey Vida Mezcal,Del Maguey Vida San Luis Del Rio Mezcal is the...,$37.99,750.0ml,None,None,4.8,42%,Mexico,https://drizly.com/liquor/mezcal/del-maguey-vi...,https://products1.imgix.drizly.com/ci-del-magu...,5.0,"Dec 6, 2018",Eric,This is a wonderful Mezcal for the price. Very...,verified buyer,"Aug 26, 2018",Alexander,I love this mezcal. Cant beat the price/taste ...,verified buyer,"Apr 11, 2018",NaN,Mezcal is pretty overpriced; the other reviews...,NaN,"Mar 30, 2018",Laralyn M.,While there are certainly nicer mezcales out t...,verified buyer,"May 8, 2017",Brittany,"Reliable mezcal, nice and smoky flavor, good p...",verified buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Total wine 

In [333]:
with open("total_wine_no_rev.pkl", "rb") as fp:  
    total_wine = pickle.load(fp)
total_wine.drop(columns=['origin'], inplace=True)
total_wine.head(1)

,Bottle Name,Brand_TW,Bottle ml_TW,Price_TW,Bottle Link_TW,Image_TW,Taste_TW
0,CASAMIGOS MEZCAL JOVEN,Casamigos,750ml,$57.99,https://www.totalwine.com/spirits/mezcal/casam...,"https://www.totalwine.com/dynamic/x490,sq/medi...",NaN


### Missionliquor

In [334]:
with open("missionliquor_no_rev.pkl", "rb") as fp:  
    missionliquor = pickle.load(fp)
missionliquor.drop(columns=['origin'], inplace=True)
missionliquor.head(1)

,Bottle Name,Price_ML,Bottle Link_ML,Image_ML
0,Alipus Mezcal San Andres 750ml,$49.95,https://www.missionliquor.com/collections/spir...,https://cdn.shopify.com/s/files/1/0144/7703/35...


### Laeuropea

In [335]:
with open("laeuropea_no_rev.pkl", "rb") as fp:  
    laeuropea = pickle.load(fp)
laeuropea.drop(columns=['origin'], inplace=True)

In [336]:
cols = ['Bottle Name', 'Price_LE', 'Bottle_Volume_LE', 'ABV_LE','Agave_LE', 'View', 'Odor', 'Taste', 'Pairing',
        'Region_LE', 'Bottle Link_LE', 'Image_LE']
laeuropea = laeuropea[cols]
laeuropea.head(1)

,Bottle Name,Price_LE,Bottle_Volume_LE,ABV_LE,Agave_LE,View,Odor,Taste,Pairing,Region_LE,Bottle Link_LE,Image_LE
1,Mezcal Gusano Rojo,422.01,750,38,NaN,Dorado,Aroma suave,Sabor único y puro que evoca las tradiciones d...,NaN,Oaxaca,https://www.laeuropea.com.mx/productos/mezcal-...,https://www.laeuropea.com.mx/img/products/2213...


### Bodega

In [337]:
with open("bodega_no_rev.pkl", "rb") as fp:  
    bodega = pickle.load(fp)
bodega.drop(columns=['origin'], inplace=True)
cols = ['Bottle Name', 'Price_BD', 'Bottle Volume_BD', 'Description', 'ABV_BD', 'Bottle Link_BD', 'Image_BD']
bodega = bodega[cols]
bodega.head(1)

,Bottle Name,Price_BD,Bottle Volume_BD,Description,ABV_BD,Bottle Link_BD,Image_BD
0,Mezcal Montelobos Joven Espadin,565.0,750,Mezcal Montelobos Joven Espadin 750 ml. Toma s...,43.20,https://www.bodegasalianza.com/mezcal-montelob...,https://bodegasalianza.vteximg.com.br/arquivos...


In [338]:
for i in bodega.index:
    descr = bodega['Description'].loc[i]
    if type(descr) == str:
        descr = re.sub('\n+|\r+', '', descr)
        descr = re.sub('•', ' ', descr)
        bodega['Description'].loc[i] = descr

### Mercadode

In [339]:
with open("mercadode_no_rev.pkl", "rb") as fp:  
    mercadode = pickle.load(fp)
mercadode.drop(columns=['origin'], inplace=True)

In [340]:
cols = [ 'Bottle Name', 'Price_MD', 'Description','Brand_MD', 'Bottle Volume_MD','Image_MD','Bottle_Link_MD']
mercadode['Description'] = mercadode['Description'].str.replace(r'(No product description or technical.+)','')
mercadode = mercadode[cols]
mercadode.head(1)

,Bottle Name,Price_MD,Description,Brand_MD,Bottle Volume_MD,Image_MD,Bottle_Link_MD
0,Agave de Cortes Blanco,44.99,,Agave de Cortes Mezcal,750,https://cdn11.bigcommerce.com/s-rhsywxdtjf/ima...,https://ca.mercadodemezcal.com/agave-de-cortes...


### Processing names

In [166]:
for table in [mezcal_rev, old_tequila, drizly, mezcalia, total_wine, missionliquor, laeuropea, bodega, mercadode]:
    table['Bottle Name'] = table['Bottle Name'].str.lower().str.replace(r'(mezcal )','').str.replace(r'( mezcal)','')
    table['Bottle Name'] = table['Bottle Name'].str.replace(r' \d\d\dml | \d\d\d ml','').str.replace(r' \d\d\dml|\d\d\d ML','')
    table['Bottle Name'] = table['Bottle Name'].str.replace(r' \d\d\dML| \d\d\dmL','').str.replace(r' \(\d\d\dML\)| \(\d\d\d ML\)','')

    table['Bottle Name'] = table['Bottle Name'].str.replace(r'( +)',' ').str.replace(r'( $|^ )','')
    for idx in table.index:
        table['Bottle Name'].loc[idx] = ' '.join([word[0].upper() + word[1:]  for word in str(table['Bottle Name'].loc[idx]).lower().split(' ')])

# To GS

In [162]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-sheets-270906-86a657daa9f6.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1KBlFSr2JDLz13XLduYzLvhMJNqYbUiyjzffmzNjghnw'

In [163]:
def to_gs(data, sheet):
    d2g.upload(data,
               spreadsheet_key,
               sheet,
               credentials=credentials,
               col_names=True,
               row_names=False,
               start_cell = 'A1',
               clean=False)

#### Mercadode

In [344]:
mercadode = mercadode.dropna(subset=['Bottle Name']).fillna('')
with open("mercadode_to_gs.pkl", "wb") as fp:  
    pickle.dump(mercadode, fp)
to_gs(mercadode, 'Mercadode')
mercadode.shape

(294, 7)

#### Bodega

In [345]:
bodega = bodega.dropna(subset=['Bottle Name']).fillna('').copy()
with open("bodega_to_gs.pkl", "wb") as fp:  
    pickle.dump(bodega, fp)
to_gs(bodega, 'Bodega')
bodega.shape

(99, 7)

#### Laeuropea

In [346]:
laeuropea = laeuropea.dropna(subset=['Bottle Name']).fillna('').copy()
with open("laeuropea_to_gs.pkl", "wb") as fp:  
    pickle.dump(laeuropea, fp)
to_gs(laeuropea, 'Laeuropea')
laeuropea.shape

(98, 12)

#### Missionliquor

In [347]:
missionliquor = missionliquor.dropna(subset=['Bottle Name']).fillna('').copy()
with open("missionliquor_to_gs.pkl", "wb") as fp:  
    pickle.dump(missionliquor, fp)
to_gs(missionliquor, 'Missionliquor')
missionliquor.shape

(283, 4)

#### Total wine

In [348]:
total_wine = total_wine.dropna(subset=['Bottle Name']).fillna('').copy()
with open("total_wine_to_gs.pkl", "wb") as fp:  
    pickle.dump(total_wine, fp)
to_gs(total_wine, 'Total Wine')
total_wine.shape

(76, 7)

#### Drizly

In [349]:
drizly = drizly.dropna(subset=['Bottle Name']).fillna('').copy()
with open("drizly_to_gs.pkl", "wb") as fp:  
    pickle.dump(drizly, fp)
to_gs(drizly, 'Drizly')
drizly.shape

(300, 52)

#### Mezcalia

In [350]:
mezcalia = mezcalia.dropna(subset=['Bottle Name']).fillna('').copy()
with open("mezcalia_to_gs.pkl", "wb") as fp:  
    pickle.dump(mezcalia, fp)
to_gs(mezcalia, 'Mezcalia')
mezcalia.shape

(221, 14)

#### Old tequila

In [167]:
old_tequila = old_tequila.dropna(subset=['Bottle Name']).fillna('').copy()
with open("old_tequila_to_gs.pkl", "wb") as fp:  
    pickle.dump(old_tequila, fp)
to_gs(old_tequila, 'Old Tequila')
old_tequila.shape

(632, 35)

#### Mezcal_rev

In [352]:
mezcal_rev = mezcal_rev.dropna(subset=['Bottle Name']).fillna('').copy()
with open("mezcal_rev_to_gs.pkl", "wb") as fp:  
    pickle.dump(mezcal_rev, fp)
to_gs(mezcal_rev, 'Mezcal Reviews')
mezcal_rev.shape

(831, 16)